In [12]:
%pip install nltk
%pip install tweepy
%pip install pandas
%pip install wordcloud
%pip install textblob
%pip install seaborn
%pip install poyraz
%pip install spacy
%pip install sklearn
%pip install zemberek-python
%pip install trnltk

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated pa

ERROR: Could not find a version that satisfies the requirement poyraz (from versions: none)
ERROR: No matching distribution found for poyraz


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
     -------------------------------------- 161.1/161.1 kB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for trnltk: filename=TRNLTK-0.0.1-py3-none-any.whl size=283895 sha256=a36dee0c94330b3a29978fb4436f4eb692d3c63db36938b6fe5ceb1ddac59c01
  Stored in directory: c:\users\cemke\appdata\local\pip\cache\wheels\47\1a\c8\221b13b08daf030ce61b722fff84d952939da7c8b0bc687454
Successfully built trnltk
No

In [13]:
#!/usr/bin/env python
#-*-coding:utf-8-*-
from zemberek import (
    TurkishSpellChecker,
    TurkishSentenceNormalizer,
    TurkishSentenceExtractor,
    TurkishMorphology,
    TurkishTokenizer
)
import logging
import tweepy
import json
import csv
import sys
import time
import re
import warnings
import string
import pandas as pd
import ast
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.porter import PorterStemmer
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import Stream
from wordcloud import WordCloud
from textblob import TextBlob
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.metrics import f1_score
from spacy.tokenizer import Tokenizer
from spacy.lang.tr import Turkish
from trnlp import TrnlpWord
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\cemke\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [20]:
class getTweets:
    def __init__(self,ckey,csecret,atoken,asecret):
        self.api=None
        self.consumer_key = ckey
        self.consumer_secret = csecret
        self.access_token = atoken
        self.access_token_secret = asecret
        
    def setAuth(self):
        try:
            self.auth = tweepy.OAuthHandler(self.consumer_key, self.consumer_secret)
            self.auth.set_access_token(self.access_token, self.access_token_secret)
            self.api = tweepy.API(self.auth)            
            print ("Bağlantı Yapıldı!")                           
        except tweepy.TweepError as err:
            print('Error: {}'.format(err))   
    
    def toDataFrame(tweets):
        try:
            print ("Veriler Kayıt Ediliyor...")
            pd.set_option('display.max_rows', None)
            pd.set_option('display.max_columns', None)
            pd.set_option('display.width', None)
            pd.set_option('display.max_colwidth', None)
            DataSet = pd.DataFrame()

            #DataSet['Tweet_Id'] = [tweet.id for tweet in tweets]
            DataSet['Tweet'] = [tweet.text for tweet in tweets]
            DataSet['Tweet_Degistirilmis'] = [tweet.text for tweet in tweets]
            DataSet['Tweet_Ceviri']=[tweet.text for tweet in tweets]
            DataSet['Tweet_Tarih'] = [tweet.created_at for tweet in tweets]
            #DataSet['Kullanıcı_Id'] = [tweet.user.id for tweet in tweets]  
            DataSet['Kullanıcı_Adı'] = [tweet.user.name for tweet in tweets]       
            #DataSet['Dil'] = [tweet.lang for tweet in tweets]
            print ("Veriler Kayıt Edildi!")
            return DataSet
        except Exception as err:
            print('Error: {}'.format(err))
    
    def searchTweet(self,aranan):
        self.setAuth()
        try:                      
            print ("Kelimeye Uygun Tweetler Sorgulanıyor...")
            cursor = tweepy.Cursor(self.api.search_tweets, q=aranan, result_type="new", lang='tr')
            results=[]
            for item in cursor.items(10):
                results.append(item)
            print ("Kelimeye Uygun Tweetler Sorgulandı!")
            DataSet = getTweets.toDataFrame(results)  
            getTweets.TweetClear(DataSet)
            getTweets.SentenceNormalization(DataSet)
            getTweets.TweetTranslate(DataSet)
            getTweets.KelimeAyristirma(DataSet) 
            DataSet.to_csv('tw.csv',index=False)
        except Exception as err:
            print('Error: {}'.format(err))
        finally:
            print ("Duygu Analizi Yapılıyor...")
            Veri=pd.read_csv('tw.csv')
            Veri["Tür"] = Veri["Tweet_Ayrı"].apply(getTweets.getPolarity)
            Veri["Yoğunluk"] = Veri["Tweet_Ayrı"].apply(getTweets.getSubjectivity)
            Veri.to_csv('textblob_eng_duygu.csv',index=False)
            print("Duygu Analizi Yapıldı!")
                  
    def TweetTranslate(DataSet):
        try:
            print ("Tweetler Çeviriliyor...")
            for yazi in range(len(DataSet["Tweet_Degistirilmis"])):
                Ceviri_Blob=TextBlob(DataSet["Tweet_Degistirilmis"][yazi])
                Eng_Blob=Ceviri_Blob.translate(from_lang='tr', to='en')  
                DataSet["Tweet_Ceviri"][yazi]=Eng_Blob
                DataSet["Tweet_Ceviri"][yazi]=DataSet["Tweet_Ceviri"][yazi].lower()
            print ("Tweetler Çevirildi!")                      
            return DataSet        
        except Exception as err:
            print('Error: {}'.format(err))
            
    def TweetClear(DataSet):
        try:
            print ("Tweetler Temizleniyor...")
            pattern = re.compile('http[s]?://(?:[A-Za-z0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+|#[A-Za-z0-9]+|$[A-Za-z0-9]+|@[A-Za-z0-9_]+|[,.^_$*%-;鶯!?’"“”:=|…]+|\n+|RT+|&amp')
            for i in range(len(DataSet["Tweet_Degistirilmis"])):        
                DataSet["Tweet_Degistirilmis"][i]=pattern.sub('', DataSet["Tweet_Degistirilmis"][i])    
                DataSet["Tweet_Degistirilmis"][i]=DataSet["Tweet_Degistirilmis"][i].lower()              
            DataSet["Tweet_Degistirilmis"]=DataSet["Tweet_Degistirilmis"].str.replace("[^\w\s]","")
            print ("Tweetler Temizlendi!")
            return DataSet
        except Exception as err:
            print('Error: {}'.format(err))
            
    def SentenceNormalization(DataSet):
        try:
            morphology = TurkishMorphology.create_with_defaults()
            normalizer = TurkishSentenceNormalizer(morphology)

            start = time.time()
            for example,index in DataSet["Tweet_Degistirilmis"]:
                 DataSet["Tweet_Degistirilmis"][index]= normalizer.normalize(example)
            return DataSet
        except Exception as err:
            print('Error: {}'.format(err))
            
    def KelimeAyristirma(DataSet):
        try:
            stop = stopwords.words('english')
            DataSet["Tweet_Ayrı"] = DataSet["Tweet_Ceviri"].str.split()
            DataSet["Tweet_Ayrı"] = DataSet["Tweet_Ayrı"].apply(lambda x: [item for item in x if item not in stop])
            DataSet["Tweet_Ayrı"] = DataSet["Tweet_Ayrı"].str.join(" ")
            return DataSet
        except Exception as err:
            print('Error: {}'.format(err))
        
    def getSubjectivity(DataSet):
        try: 
            sub=TextBlob(DataSet).sentiment.subjectivity               
            if sub >=0.75:
                return "Kesinlikle"
            elif sub >=0.5 and sub <0.75:
                return "Yüksek İhtimal"
            elif sub >=0.25 and sub <0.50:
                return "Düşük İhtimal"
            else:
                return "Kesin Değil"
        except Exception as err:
            print('Error: {}'.format(err))
            
    def getPolarity(DataSet):
        try:
            pol=TextBlob(DataSet).sentiment.polarity
            if pol>0.1:
                return "Pozitif"
            elif pol<-0.1:
                return "Negatif"
            else:
                return "Nötr"
        except Exception as err:
            print('Error: {}'.format(err))

In [21]:
tw = getTweets( 
ckey = 'o7qidRODRWm5u07PtAWrCpSqf',
csecret = '1Yn60OYMJl5kRCSM0XQ4Lskxh6hjM4VMz51OdsS6dM9DQuQ4MV',
atoken = '1315792402234441730-Dt7AydgVGYI5VN9RRQfnR8Endh0WAc',
asecret = 'jhfiDfB53Z27EqHfvfKbNdo0w2SWLqyyoAkILqgSoaGMK'
)
aranan=input("Aranacak Kelimeyi Girin: ")
tw.searchTweet(aranan)

Bağlantı Yapıldı!
Kelimeye Uygun Tweetler Sorgulanıyor...
Kelimeye Uygun Tweetler Sorgulandı!
Veriler Kayıt Ediliyor...
Veriler Kayıt Edildi!
Tweetler Temizleniyor...
Tweetler Temizlendi!
2023-01-09 21:50:42,097 - zemberek.morphology.turkish_morphology - INFO
Msg: TurkishMorphology instance initialized in 8.600984811782837

Error: too many values to unpack (expected 2)
Tweetler Çeviriliyor...
Tweetler Çevirildi!
Duygu Analizi Yapılıyor...
Duygu Analizi Yapıldı!


In [22]:
Veri=pd.read_csv('textblob_eng_duygu.csv')
Veri.head()

,Tweet,Tweet_Degistirilmis,Tweet_Ceviri,Tweet_Tarih,Kullanıcı_Adı,Tweet_Ayrı,Tür,Yoğunluk
0,@eczaylin Aşırı zorlamaya bağlı eklem aralığındaki sıvı kapsülü ile bir çeşit fiktiklasiyor. \nBandaj iyi geliyor. G… https://t.co/YW4GfHbJCb,aşırı zorlamaya bağlı eklem aralığındaki sıvı kapsülü ile bir çeşit fiktiklasiyor bandaj iyi geliyor g,"with the liquid capsule in the joint range due to excessive coercion, some kind of bandage is good g",2023-01-09 18:50:29+00:00,Dr.Hakan Akelma 🩺,"liquid capsule joint range due excessive coercion, kind bandage good g",Pozitif,Yüksek İhtimal
1,"#karsiyaka 1-)%&amp; çiğli + Bertino şaşkınlık ve dehşet içinde suskun kaldı. Onun iyi huylu, https://t.co/EUaUwbD5CO",amp çiğli bertino şaşkınlık ve dehşet içinde suskun kaldı onun iyi huylu,amp çiğli bertino remained silent in astonishment and horror,2023-01-09 18:50:29+00:00,njamenmedipeh,amp çiğli bertino remained silent astonishment horror,Nötr,Kesin Değil
2,"Sanırım dayak istiyor. Kız iyi bi öğrenci ama tabi ki kesinlikle düzeltecek bu ödevi, çizdim geçtim o salak paragra… https://t.co/FCwnilAIsf",sanırım dayak istiyor kız iyi bi öğrenci ama tabi ki kesinlikle düzeltecek bu ödevi çizdim geçtim o salak paragra,"i think the girl wants a good student, but of course i drew this homework that will definitely correct it, i passed that stupid paragra",2023-01-09 18:50:28+00:00,tugba,"think girl wants good student, course drew homework definitely correct it, passed stupid paragra",Nötr,Yüksek İhtimal


In [23]:
Veri1=pd.read_csv('tw.csv')
Veri1.head()

,Tweet,Tweet_Degistirilmis,Tweet_Ceviri,Tweet_Tarih,Kullanıcı_Adı,Tweet_Ayrı
0,@eczaylin Aşırı zorlamaya bağlı eklem aralığındaki sıvı kapsülü ile bir çeşit fiktiklasiyor. \nBandaj iyi geliyor. G… https://t.co/YW4GfHbJCb,aşırı zorlamaya bağlı eklem aralığındaki sıvı kapsülü ile bir çeşit fiktiklasiyor bandaj iyi geliyor g,"with the liquid capsule in the joint range due to excessive coercion, some kind of bandage is good g",2023-01-09 18:50:29+00:00,Dr.Hakan Akelma 🩺,"liquid capsule joint range due excessive coercion, kind bandage good g"
1,"#karsiyaka 1-)%&amp; çiğli + Bertino şaşkınlık ve dehşet içinde suskun kaldı. Onun iyi huylu, https://t.co/EUaUwbD5CO",amp çiğli bertino şaşkınlık ve dehşet içinde suskun kaldı onun iyi huylu,amp çiğli bertino remained silent in astonishment and horror,2023-01-09 18:50:29+00:00,njamenmedipeh,amp çiğli bertino remained silent astonishment horror
2,"Sanırım dayak istiyor. Kız iyi bi öğrenci ama tabi ki kesinlikle düzeltecek bu ödevi, çizdim geçtim o salak paragra… https://t.co/FCwnilAIsf",sanırım dayak istiyor kız iyi bi öğrenci ama tabi ki kesinlikle düzeltecek bu ödevi çizdim geçtim o salak paragra,"i think the girl wants a good student, but of course i drew this homework that will definitely correct it, i passed that stupid paragra",2023-01-09 18:50:28+00:00,tugba,"think girl wants good student, course drew homework definitely correct it, passed stupid paragra"
